Download labeling from GitHub - https://github.com/tzutalin/labelImg


`!pip install pyqt5`

`!pip install lxml`


Installation guide - https://github.com/heartexlabs/labelImg#installation

Run this command in the terminal

`pyrcc5 -o libs/resources.py resources.qrc`

In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.retinanet import RetinaNet_ResNet50_FPN_Weights

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Number of classes including background
num_classes = 91 

# Load a pre-trained model
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True, num_classes=num_classes)

model.to(device)

# Continue with your code... (Origional numbers)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)


class YOLODataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform if transform else transforms.ToTensor()
        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "images"))))
        self.labels = list(sorted(os.listdir(os.path.join(root_dir, "labels"))))

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, "images", self.imgs[idx])
        label_path = os.path.join(self.root_dir, "labels", self.labels[idx])

        img = Image.open(img_path).convert("RGB")
        img_width, img_height = img.size

        # Apply transformation after getting original size
        img = self.transform(img)

        # Read YOLO label file
        with open(label_path, "r") as file:
            lines = file.read().splitlines()

        boxes = []
        labels = []
        for line in lines:
            class_id, x_center, y_center, width, height = map(float, line.split())
            labels.append(int(class_id))

            x_min = img_width * (x_center - width / 2)
            y_min = img_height * (y_center - height / 2)
            x_max = img_width * (x_center + width / 2)
            y_max = img_height * (y_center + height / 2)
            boxes.append([x_min, y_min, x_max, y_max])

        target = {}
        target['boxes'] = torch.tensor(boxes, dtype=torch.float32)
        target['labels'] = torch.tensor(labels, dtype=torch.int64)

        return img, target

# Define your own paths here
train_dataset = YOLODataset("data/train set")
valid_dataset = YOLODataset("data/validation set")

def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]

    return images, targets

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.5, weight_decay=0.0005)


c:\Users\dalto\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\dalto\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
model.train()
num_epochs = 500
model = model.to(device)
for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_dataloader):
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]


        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch: {epoch + 1}, Loss: {losses.item()}")

Epoch: 1, Loss: 0.29519328474998474
Epoch: 2, Loss: 0.23080462217330933
Epoch: 3, Loss: 0.2574554681777954
Epoch: 4, Loss: 0.20284490287303925
Epoch: 5, Loss: 0.19133850932121277
Epoch: 6, Loss: 0.17199423909187317
Epoch: 7, Loss: 0.1196833923459053
Epoch: 8, Loss: 0.21816259622573853
Epoch: 9, Loss: 0.12178077548742294
Epoch: 10, Loss: 0.10643225908279419
Epoch: 11, Loss: 0.1746397614479065
Epoch: 12, Loss: 0.0936783105134964
Epoch: 13, Loss: 0.1043352484703064
Epoch: 14, Loss: 0.15091785788536072
Epoch: 15, Loss: 0.13693469762802124
Epoch: 16, Loss: 0.08447380363941193
Epoch: 17, Loss: 0.12717850506305695
Epoch: 18, Loss: 0.12243068963289261
Epoch: 19, Loss: 0.12133996188640594
Epoch: 20, Loss: 0.13260038197040558
Epoch: 21, Loss: 0.11567588895559311
Epoch: 22, Loss: 0.14327619969844818
Epoch: 23, Loss: 0.10531725734472275
Epoch: 24, Loss: 0.1216469556093216
Epoch: 25, Loss: 0.1006377711892128
Epoch: 26, Loss: 0.07615561783313751
Epoch: 27, Loss: 0.0701209008693695
Epoch: 28, Loss: 0